In [562]:
from cmath import nan
from tkinter import Y
import unittest
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, date
from datetime import timedelta
import os
import csv
from collections import defaultdict
from dateutil.relativedelta import relativedelta
from collections import defaultdict
from collections import ChainMap

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


#### Importing MC4R data as .csv to a dataframe

In [563]:
mc4r_data_df = pd.read_csv('/Users/adam/Documents/BIH/Adipositas/MC4R/Original MC4R Data/MC4R_Phaeno_Demographics.csv', skipinitialspace=True, sep=",", decimal=',')
mc4r_gendata_df = pd.read_csv('/Users/adam/Documents/BIH/Adipositas/MC4R/HGVS Standardisierung/HGVS_Standardisierung_revised.csv', skipinitialspace=True, sep=",", decimal=',').replace({pd.NA: ''})
mc4r_weightcourse_df = pd.read_csv('/Users/adam/Documents/BIH/Adipositas/MC4R/Original MC4R Data/mc4r_weight_course_revised.csv', skipinitialspace=True, sep=",", decimal=',')

### Mapping MC4R data toERKER format

#### 1. Formale Kriterien

In [564]:
# Record ID for REDCap 
mc4r_data_df['record_id'] = mc4r_data_df.index +1

#### 2. Persönliche Angaben

In [565]:
# 2.1 Geburtsjahr [JJJJ]
mc4r_data_df['Date of Birth DD/MM/YYYY'] = pd.to_datetime(mc4r_data_df['Date of Birth DD/MM/YYYY'], format='%Y-%m-%d') 

birth_date = pd.to_datetime(mc4r_data_df['Date of Birth DD/MM/YYYY'], format='%Y-%m-%d')
birth_date_y = pd.DatetimeIndex(mc4r_data_df['Date of Birth DD/MM/YYYY']).year
birth_date_m = pd.DatetimeIndex(mc4r_data_df['Date of Birth DD/MM/YYYY']).month
birth_date_d = pd.DatetimeIndex(mc4r_data_df['Date of Birth DD/MM/YYYY']).day

def age(born):
    born = datetime.strptime(str(born), "%Y-%m-%d").date()
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

# 2.3 Administratives Geschlecht
dict_sct_263495000 = {'f' : 'sct_248152002', 'm' : 'sct_248153007'}

# 2.4 Ethnizität
dict_sct_397731000 = {'deutsch':  'sct_14045001',
                    'polnisch': 'sct_14045001',
                    'deutsch':  'sct_14045001', 
                    'türkisch': 'sct_14045001',
                    'russisch': 'sct_14045001',
                    'bosnisch': 'sct_14045001',
                    'serbisch': 'sct_14045001',
                    'bulgarisch': 'sct_14045001',
                    'jugoslawisch': 'sct_14045001',
                     
                    'deutsch/italienisch': 'sct_372148003_246261001_26242008',
                    'deutsch/türkisch': 'sct_372148003_246261001_26242008',
                    'türkisch/deutsch': 'sct_372148003_246261001_26242008',
                    'deutsch/arabisch': 'sct_372148003_246261001_26242008',
                    'ukrainisch/aserbaidschanisch': 'sct_372148003_246261001_26242008',
                    'deutsch/armenisch': 'sct_372148003_246261001_26242008',
                    'deutsch/pakistanisch': 'sct_372148003_246261001_26242008',
                    'ägyptisch/deutsch': 'sct_372148003_246261001_26242008',
                    'deutsch/beninisch': 'sct_372148003_246261001_26242008',
                    'deutsch/albanisch': 'sct_372148003_246261001_26242008',
                    'russisch/deutsch': 'sct_372148003_246261001_26242008',
                    'deutsch/nigerianisch': 'sct_372148003_246261001_26242008',
                    'deutsch/serbisch': 'sct_372148003_246261001_26242008',

                    'irakisch': 'sct_90027003',
                    'arabisch': 'sct_90027003',
                    'syrisch': 'sct_90027003',
                     
                    'brasilianisch': 'sct_414408004', 
                    nan : ''
                    }



#### 3. Patientenstatus

In [566]:
mc4r_agecat_df = pd.DataFrame()
mc4r_agecat_df['Age'] = mc4r_data_df['Date of Birth DD/MM/YYYY'].apply(lambda x: age(x.to_pydatetime().date()))

dict_age_category= {
    pd.Interval(0.0, 1.0, closed='left'): 'sct_3658006', #'Säugling'
    pd.Interval(1.0, 6.0, closed='left'): 'sct_713153009', #'Kleinkind'
    pd.Interval(6.0, 12.0, closed='left'): 'sct_255398004', #'Schulkind',
    pd.Interval(12.0, 18.0, closed='left'): 'sct_263659003', #'Jugendliche'
    pd.Interval(18.0, np.inf, closed='left'): 'sct_41847000' #'Erwachsen
}

mc4r_agecat_df['Age Category'] = mc4r_agecat_df['Age'].map(dict_age_category)
#print(mc4r_agecat_df['Age Category'])
def AgeCategory(row):
    for row in mc4r_data_df['Date of Birth DD/MM/YYYY'].apply(lambda x: age(x.to_pydatetime().date())).map(dic_age_category):
        pass

#### 4. Behandlungsverlauf

###### 4.1 Erstkontakt mit dem Fachzentrum

In [567]:
erstkontakt_y = mc4r_data_df['Age of Presentation\n0Y 0M'].str[:2].astype(int)
erstkontakt_m = mc4r_data_df['Age of Presentation\n0Y 0M'].str[3:5].astype(int)
erstkontakt_date_y = (birth_date_y + erstkontakt_y).astype('string')
erstkontakt_date_m = (birth_date_m + erstkontakt_m).astype('string')

erstkontakt_date = erstkontakt_date_y + "-" + erstkontakt_date_m + "-" + birth_date_d.astype('string')
mc4r_erstkontakt_df = pd.DataFrame()
mc4r_erstkontakt_df['Erstkontakt'] = erstkontakt_date

#### 5. Krankheitsverlauf

In [568]:
#5.2 Alter bei Diagnose
date_diagnosis = mc4r_data_df['Date of diagnosis\nDD/MM/YYYY'].astype('string').fillna(np.nan)
date_diagnosis_df = pd.DataFrame()
date_diagnosis_df['DateDiagnosis'] = date_diagnosis
age_at_diagnosis = date_diagnosis_df.isnull()
age_at_diagnosis_df = pd.DataFrame()
age_at_diagnosis_df['nan_sct_423493009'] = age_at_diagnosis
aad_nan_dict = {False: 'sct_410672004', True: 'sct_261665006'}
date_diagnosis_df['nan_date_diagnosis'] = pd.isna(date_diagnosis_df['DateDiagnosis']).map(aad_nan_dict)
date_diagnosis_df = date_diagnosis_df.replace({pd.NA: ''})
age_at_diagnosis_df['nan_date_diagnosis'] = date_diagnosis_df['nan_date_diagnosis']
age_at_diagnosis_df['date_diagnosis_y'] = date_diagnosis.str[:4].replace({pd.NA: ''})
age_at_diagnosis_df['date_diagnosis_m'] = date_diagnosis.str[5:7].replace({pd.NA: ''})
age_at_diagnosis_df['date_diagnosis_d'] = date_diagnosis.str[8:10].replace({pd.NA: ''})


#5.7 Konsanguinität
consanguinity_df = pd.DataFrame(mc4r_data_df['Consanguinity\nY/N'])
consanguinity_df_dict = {'y' : 'sct_373066001',
                      'n' : 'sct_373067005'
                     }
consanguinity_df['consanguinity'] = mc4r_data_df['Consanguinity\nY/N'].map(consanguinity_df_dict)
consanguinity_df = consanguinity_df.replace({pd.NA: 'sct_1220561009'})

#### 6.1 Hauptdiagnose

In [569]:
# 6.1.9 Sporadisches oder familiäres Auftreten?
occurrence_df = pd.DataFrame(mc4r_data_df['Familiy history of obesity\nY/N'])
occurrence_df_dic = {'y': 'sct_255401001'}
occurrence_df['occurrence'] = mc4r_data_df['Familiy history of obesity\nY/N'].map(occurrence_df_dic)
occurrence_df = occurrence_df.replace({pd.NA: 'sct_1220561009'})

#### 6.2 genetischer Befund

###### sort and merge both dataframes

In [570]:
columns_to_convert = ['OMIM_g']
mc4r_gendata_df[columns_to_convert] = mc4r_gendata_df[columns_to_convert].astype(str)
mc4r_genotyping = pd.merge_asof(mc4r_gendata_df, mc4r_data_df, on='record_id')
mc4r_genotyping.head()

,record_id,HGVS Freitext,Varsome:HGVS,Freitext,c.HGVS (validiert),p.HGVS (validiert),OMIM_g,zygosity,Type of Mutation (ln_48019_4),Ursprung der Variante (ln_48002_0),ACMG (ln_ll4034_6),links,Unnamed: 12,transcript,Unnamed: 14,Unnamed: 15,Unnamed: 16,Notizen:,Patienten ID,Date of Birth DD/MM/YYYY,Age of Presentation\n0Y 0M,Sex\nm/f/d,Ethnic group\n,Mutation\n,Mutation type\n,Date of diagnosis\nDD/MM/YYYY,Familiy history of obesity\nY/N,Consanguinity\nY/N,BMI Father\nkg/m2,BMI Mother\nkg/m2,1.Sibling BMI\nkg/m2,2. Sibling BMI\nkg/m2,3. Sibling BMI\nkg/m2,Concomitant Medication,Comorbidities,(History of) Hyperphagia,Unnamed: 18
0,1,V103l,MC4R(NM_005912.3):c.307G>A\n(p.Val103Ile),chr18-60372043-C-T,NM_005912.3:c.307G>A,NP_005903.2:p.(Val103Ile),155541.0024,ln_LA6706-1,SO_1000002,sct_1220561009,ln_LA26333-7,https://varsome.com/variant/hg38/MC4R%3AV103L?...,NaN,NaN,NaN,NaN,NaN,https://lhcforms.nlm.nih.gov/fhir/hgvs-validator/,FA062,2001-08-24,06/05,m,irakisch,V103l,het. Polym.,2008-01-23,NaN,y,"24,6","26,4","22,7",17,0,0,0,Y/N,NaN
1,2,p.V103l,MC4R(NM_005912.3):c.307G>A\n(p.Val103Ile),chr18-60372043-C-T,NM_005912.3:c.307G>A,NP_005903.2:p.(Val103Ile),155541.0024,ln_LA6706-1,SO_1000002,sct_1220561009,ln_LA26333-7,https://varsome.com/variant/hg38/MC4R%3AV103L?...,NaN,NaN,NaN,NaN,NaN,,YA074,2002-08-21,09/02,f,türkisch/deutsch,p.V103l,het.,2011-04-18,NaN,0,"30,3",32,"22,6","22,4","21,8",0,0,0,NaN
2,3,,,,,,,,,,,,NaN,NaN,NaN,NaN,NaN,,SA099,2002-07-23,00/00,m,deutsch,NaN,NaN,NaN,NaN,0,"26,8","26,8",0,0,0,0,0,0,kein Genetikbefund
3,4,"c.380C>T, p.Ser127Leu",MC4R(NM_005912.3):c.380C>T\n(p.Ser127Leu),chr18-60371970-G-A,NM_005912.3:c.380C>T,NP_005903.2:p.(Ser127Leu),155541.0021,ln_LA6706-1,SO_1000002,sct_1220561009,ln_LA6668-3,https://varsome.com/variant/hg38/MC4R%3Ac.380C...,NaN,NaN,NaN,NaN,NaN,,EA102,2007-01-26,11/08,f,türkisch/deutsch,"c.380C>T, p.Ser127Leu",het.,2018-10-17,NaN,n,"27,7","34,4",0,0,0,0,0,0,NaN
4,5,T11l,,,,,,,,,,,NaN,NaN,NaN,NaN,NaN,excluded from data set,AA070,2000-02-17,09/02,f,deutsch/italienisch,T11l,het.,2009-07-05,NaN,0,"26,8","22,1","20,9",0,0,0,0,NaN,NaN


###### create columns for each genetic variant of each patient

In [571]:
def split_and_concat(df, column_name, prefix, num_columns):
    new_columns = df[column_name].str.split(';', expand=True)
    new_columns.columns = [f'{prefix}_{i+1}' for i in range(num_columns)]
    return pd.concat([df, new_columns], axis=1)

columns_to_process = [
    ('HGVS Freitext', 'hgvsfreitext', 3),
    ('c.HGVS (validiert)', 'c.HGVS', 3),
    ('p.HGVS (validiert)', 'p.HGVS', 3),
    ('OMIM_g', 'OMIM_g', 2),
    ('zygosity', 'zygosity', 3),
    ('Type of Mutation (ln_48019_4)', 'typeofmutation', 3),
    ('Ursprung der Variante (ln_48002_0)', 'ursprungdervariante', 3),
    ('ACMG (ln_ll4034_6)', 'acmg', 3)
]

for column_name, prefix, num_columns in columns_to_process:
    mc4r_genotyping = split_and_concat(mc4r_genotyping, column_name, prefix, num_columns)

#### 6.5 Phänotypisierung - Obesity

##### Creating a new dataframe for phenotyping called mc4r_phenotyping
- sorted by 'record_id' and merging record_id, Sex and Date of Birth of the original dataframe mc4r_data_df 

In [572]:
mc4r_weightcourse = mc4r_weightcourse_df.sort_values(by='record_id')
mc4r_phenotyping = mc4r_weightcourse_df.merge(mc4r_data_df[['record_id', 'Sex\nm/f/d', 'Date of Birth DD/MM/YYYY']], on='record_id', how='left')

##### BMI to Age tables of the WHO are read as .csv files for different age and gender groups
- BMI Birth - Age 5: https://www.who.int/tools/growth-reference-data-for-5to19-years/indicators/bmi-for-agehttps://www.who.int/tools/growth-reference-data-for-5to19-years/indicators/bmi-for-age 
- BMI Age 5-19: https://www.who.int/tools/growth-reference-data-for-5to19-years/indicators/bmi-for-agehttps://www.who.int/tools/growth-reference-data-for-5to19-years/indicators/bmi-for-age

In [573]:
boys_0_2_data = pd.read_csv('/Users/adam/Documents/BIH/Adipositas/MC4R/bmi-to-age/tab_bmi_boys_p_0_2.csv',skipinitialspace=True, sep=";", decimal=',')
boys_2_5_data = pd.read_csv('/Users/adam/Documents/BIH/Adipositas/MC4R/bmi-to-age/tab_bmi_boys_p_2_5.csv',skipinitialspace=True, sep=";", decimal=',')
girls_0_2_data = pd.read_csv('/Users/adam/Documents/BIH/Adipositas/MC4R/bmi-to-age/tab_bmi_girls_p_0_2.csv',skipinitialspace=True, sep=";", decimal=',')
girls_2_5_data = pd.read_csv('/Users/adam/Documents/BIH/Adipositas/MC4R/bmi-to-age/tab_bmi_girls_p_2_5.csv',skipinitialspace=True, sep=";", decimal=',')
boys_5_19_data = pd.read_csv('/Users/adam/Documents/BIH/Adipositas/MC4R/bmi-to-age/bmi-boys-perc-who2007-exp.csv',skipinitialspace=True, sep=";", decimal=',')
girls_5_19_data = pd.read_csv('/Users/adam/Documents/BIH/Adipositas/MC4R/bmi-to-age/bmi-girls-perc-who2007-exp.csv',skipinitialspace=True, sep=";", decimal=',')

##### Create function to convert age string of original data in mc4r_weightcourse dataframe to full months

In [574]:
# Function to convert age string to months
def convert_age_column_to_months(age_str_column):
    def convert_single_age(age_str):
        try:
            years, months = map(int, age_str.split('/'))
            total_months = years * 12 + months
            return total_months
        except (ValueError, AttributeError):
            return 0

    total_months = age_str_column.apply(convert_single_age)
    return total_months

age_columns = ['Age of U6 0Y 0M', 'Age of U7 0Y 0M', 'Age of U8 0Y 0M', 'Age of U9 0Y 0M']
# Convert age columns to months
for age_column in age_columns:
    mc4r_phenotyping[age_column[:-6] + ' in Months'] = convert_age_column_to_months(mc4r_weightcourse[age_column])


##### Define function that adds months to the birthdate to generate approximate date of examination 

In [575]:
mc4r_phenotyping['Date of Birth DD/MM/YYYY'] = pd.to_datetime(mc4r_phenotyping['Date of Birth DD/MM/YYYY'])
def add_months(date, months):
    if months == 0:
        return None
    return date + relativedelta(months=months)

##### Define function to determine percentile data based on age and sex

In [576]:
def get_percentile_data(age, sex):
    if sex == 'm':
        if age <= 24:
            return boys_0_2_data
        elif age <= 60:  # Use boys_2_5_data for ages up to 5 years
            return boys_2_5_data
        else:
            return boys_5_19_data  # Use boys_5_19_data for ages 5 years and above
    elif sex == 'f':
        if age <= 24:
            return girls_0_2_data
        elif age <= 60:  # Use girls_2_5_data for ages up to 5 years
            return girls_2_5_data
        else:
            return girls_5_19_data  # Use girls_5_19_data for ages 5 years and above

##### Define function to determine class of obesity according to Hampl S. et al (2023)
- https://publications.aap.org/pediatrics/article/151/2/e2022060640/190443/Clinical-Practice-Guideline-for-the-Evaluation-and?autologincheck=redirected
- HPO Terms from https://hpo.jax.org/ 

In [577]:
def get_obesity_class(bmi, age, sex):
    percentile_data = get_percentile_data(age, sex)

    age_column_name = 'Month'
    percentile_95 = percentile_data.loc[percentile_data['Month'] == age, 'P95'].values[0]
    percentile_85 = percentile_data.loc[percentile_data['Month'] == age, 'P85'].values[0]

    class_2_threshold = 1.2 * percentile_95
    class_3_threshold = 1.4 * percentile_95

    if bmi >= class_3_threshold:
        return "HP:0025501" # Class III obesity
    elif bmi >= class_2_threshold:
        return "HP:0025500" # Class II obesity
    elif bmi >= percentile_95:
        return "HP:0025499" # Class I obesity
    elif bmi >= percentile_85:
        return "HP:0025502" # Overweight 
    elif bmi == 0:
        return 'HP:0001507' # growth abnormality, but not recorded
    elif bmi < percentile_85: 
        return 'HP:0001513' # obesity, but refuted

##### Find out obesity class for each examination:
-  iterate through all mc4r_phenotyping dataframe for each examination (birth, U6, U7, U8, U9)
-  run get_obesity_class function on each examination and its rows
- create obesity class columns for each examination accordingly
- the status of each phenotype by appending the list birth_hpo_status_list accordingly to class_obesity

###### Birth

In [578]:
birth_class_obesity_list = []
birth_hpo_status_list = []
for index, row in mc4r_phenotyping.iterrows():
    record_id = row['record_id']
    birth_bmi = row['BMI-Birth kg/m2']
    birth_age_m = 0
    sex = row['Sex\nm/f/d']
    class_obesity = get_obesity_class(birth_bmi, birth_age_m, sex)
    birth_class_obesity_list.append(class_obesity)

    if class_obesity in ['HP:0025501', 'HP:0025500', 'HP:0025499', 'HP:0025502']: # obesity class 1-3 & overweight
        birth_hpo_status_list.append('sct_410605003') # confirmed
    elif class_obesity == 'HP:0001513': # obesity
        birth_hpo_status_list.append('sct_723511001') # but refuted
    elif birth_bmi == 'HP:0004324':
        birth_hpo_status_list.append('sct_1220561009') # increased body weight, but not recorded
    else:
        birth_hpo_status_list.append('sct_1220561009') # increased body weight, but not recorded

mc4r_phenotyping['birth_class_obesity'] = birth_class_obesity_list
mc4r_phenotyping['birth_hpo_status'] = birth_hpo_status_list


def fill_birth_date(row):
    dob = row['Date of Birth DD/MM/YYYY']
    return dob.strftime('%Y-%m-%d')

mc4r_phenotyping['birth_date'] = mc4r_phenotyping.apply(fill_birth_date, axis=1)

###### U6

In [579]:
u6_class_obesity_list = []
u6_hpo_status_list = []
for index, row in mc4r_phenotyping.iterrows():
    record_id = row['record_id']
    u6_bmi = row['BMI-U6 kg/m2']
    u6_age_m = row['Age of U6 in Months']
    sex = row['Sex\nm/f/d']
    class_obesity = get_obesity_class(u6_bmi, u6_age_m, sex)
    u6_class_obesity_list.append(class_obesity)
    
    if class_obesity in ['HP:0025501', 'HP:0025500', 'HP:0025499', 'HP:0025502']: # obesity class 1-3 & overweight
        u6_hpo_status_list.append('sct_410605003') # confirmed
    elif class_obesity == 'HP:0001513': # obesity
        u6_hpo_status_list.append('sct_723511001') # but refuted
    elif class_obesity == 'HP:0004324': # increased body weight
        u6_hpo_status_list.append('sct_1220561009') # but not recorded
    else:
        u6_hpo_status_list.append('sct_1220561009') # increased body weight, but not recorded
    

mc4r_phenotyping['u6_class_obesity'] = u6_class_obesity_list
mc4r_phenotyping['u6_hpo_status'] = u6_hpo_status_list

def fill_u6_date(row):
    dob = pd.to_datetime(row['Date of Birth DD/MM/YYYY'], format='%d/%m/%Y')
    age_in_months = row['Age of U6 in Months']
    if age_in_months == 0:
        u6_date = dob + pd.DateOffset(months=12)
    else:
        u6_date = dob + pd.DateOffset(months=age_in_months)
    return u6_date.strftime('%Y-%m-%d')

mc4r_phenotyping['u6_date'] = mc4r_phenotyping.apply(fill_u6_date, axis=1)

###### U7

In [580]:
u7_class_obesity_list = []  
u7_hpo_status_list = []
for index, row in mc4r_phenotyping.iterrows():
    record_id = row['record_id']
    u7_bmi = row['BMI-U7 kg/m2']
    u7_age_m = row['Age of U7 in Months']
    sex = row['Sex\nm/f/d']
    class_obesity = get_obesity_class(u7_bmi, u7_age_m, sex)
    u7_class_obesity_list.append(class_obesity)
    
    if class_obesity in ['HP:0025501', 'HP:0025500', 'HP:0025499', 'HP:0025502']: # obesity class 1-3 & overweight
        u7_hpo_status_list.append('sct_410605003') # confirmed
    elif class_obesity == 'HP:0001513': # obesity
        u7_hpo_status_list.append('sct_723511001') # but refuted
    elif class_obesity == 'HP:0004324': # increased body weight
        u7_hpo_status_list.append('sct_1220561009') # but not recorded
    else:
        u7_hpo_status_list.append('sct_1220561009') # increased body weight, but not recorded

mc4r_phenotyping['u7_class_obesity'] = u7_class_obesity_list
mc4r_phenotyping['u7_hpo_status'] = u7_hpo_status_list

def fill_u7_date(row):
    dob = pd.to_datetime(row['Date of Birth DD/MM/YYYY'], format='%d/%m/%Y')
    age_in_months = row['Age of U7 in Months']
    if age_in_months == 0:
        u7_date = dob + pd.DateOffset(months=24)
    else:
        u7_date = dob + pd.DateOffset(months=age_in_months)
    return u7_date.strftime('%Y-%m-%d')

mc4r_phenotyping['u7_date'] = mc4r_phenotyping.apply(fill_u7_date, axis=1)

###### U8

In [581]:
u8_class_obesity_list = []
u8_hpo_status_list = []
for index, row in mc4r_phenotyping.iterrows():
    record_id = row['record_id']
    u8_bmi = row['BMI-U8 kg/m2']
    u8_age_m = row['Age of U8 in Months']
    sex = row['Sex\nm/f/d']
    class_obesity = get_obesity_class(u8_bmi, u8_age_m, sex)
    u8_class_obesity_list.append(class_obesity)
        
    if class_obesity in ['HP:0025501', 'HP:0025500', 'HP:0025499', 'HP:0025502']: # obesity class 1-3 & overweight
        u8_hpo_status_list.append('sct_410605003') # confirmed
    elif class_obesity == 'HP:0001513': # obesity
        u8_hpo_status_list.append('sct_723511001') # but refuted
    elif class_obesity == 'HP:0004324': # increased body weight
        u8_hpo_status_list.append('sct_1220561009') # but not recorded
    else:
        u8_hpo_status_list.append('sct_1220561009') # increased body weight, but not recorded

mc4r_phenotyping['u8_class_obesity'] = u8_class_obesity_list
mc4r_phenotyping['u8_hpo_status'] = u8_hpo_status_list

def fill_u8_date(row):
    dob = pd.to_datetime(row['Date of Birth DD/MM/YYYY'], format='%d/%m/%Y')
    age_in_months = row['Age of U8 in Months']
    if age_in_months == 0:
        u8_date = dob + pd.DateOffset(months=48)
    else:
        u8_date = dob + pd.DateOffset(months=age_in_months)
    return u8_date.strftime('%Y-%m-%d')

mc4r_phenotyping['u8_date'] = mc4r_phenotyping.apply(fill_u8_date, axis=1)



###### U9

In [582]:
u9_class_obesity_list = []  
u9_hpo_status_list = []
u9_date_list = []
for index, row in mc4r_phenotyping.iterrows():
    record_id = row['record_id']
    u9_bmi = row['BMI-U9 kg/m2']
    u9_age_m = row['Age of U9 in Months']
    sex = row['Sex\nm/f/d']
    class_obesity = get_obesity_class(u9_bmi, u9_age_m, sex)
    u9_class_obesity_list.append(class_obesity)
    
    if class_obesity in ['HP:0025501', 'HP:0025500', 'HP:0025499', 'HP:0025502']: # obesity class 1-3 & overweight
        u9_hpo_status_list.append('sct_410605003') # confirmed
    elif class_obesity == 'HP:0001513': # obesity
        u9_hpo_status_list.append('sct_723511001') # but refuted
    elif class_obesity == 'HP:0004324': # increased body weight
        u9_hpo_status_list.append('sct_1220561009') # but not recorded
    else:
        u9_hpo_status_list.append('sct_1220561009') # increased body weight, but not recorded

mc4r_phenotyping['u9_class_obesity'] = u9_class_obesity_list
mc4r_phenotyping['u9_hpo_status'] = u9_hpo_status_list

def fill_u9_date(row):
    dob = pd.to_datetime(row['Date of Birth DD/MM/YYYY'], format='%d/%m/%Y')
    age_in_months = row['Age of U9 in Months'] 
    if age_in_months == 0:
        u9_date = dob + pd.DateOffset(months=62)
    else:
        u9_date = dob + pd.DateOffset(months=age_in_months)
    return u9_date.strftime('%Y-%m-%d')

mc4r_phenotyping['u9_date'] = mc4r_phenotyping.apply(fill_u9_date, axis=1)


#### 6.5 Phänotypisierung - RR

In [583]:
mc4r_rr_unsorted = pd.read_csv('/Users/adam/Documents/BIH/Adipositas/MC4R/Original MC4R Data/mc4r_rr.csv', skipinitialspace=True, sep=";", decimal=',')
columns_to_drop = ['24h - RR-1 Hypertension y/n', '24h - RR-2 Hypertension y/n', 'Unnamed: 7']
mc4r_rr_unsorted = mc4r_rr_unsorted.drop(columns=columns_to_drop)
mc4r_rr_unsorted = mc4r_rr_unsorted.rename(columns={'Patienten ID ': 'Patienten ID'})

mc4r_rr = pd.merge(mc4r_rr_unsorted, mc4r_data_df, on='Patienten ID', how='inner')
mc4r_rr = mc4r_rr.sort_values(by='record_id')

col_drop = ['Age of Presentation\n0Y 0M', 'Ethnic group\n', 'Mutation\n','Mutation type\n', 'Date of diagnosis\nDD/MM/YYYY', 
            'Familiy history of obesity\nY/N', 'Consanguinity\nY/N', 'BMI Father\nkg/m2', '1.Sibling BMI\nkg/m2', '2. Sibling BMI\nkg/m2', 
            '3. Sibling BMI\nkg/m2', 'Concomitant Medication', 'Comorbidities', '(History of) Hyperphagia', 'Unnamed: 18']
mc4r_rr = mc4r_rr.drop(columns = col_drop)

mc4r_rr = mc4r_rr[['record_id'] + [col for col in mc4r_rr if col != 'record_id']]

In [584]:
mc4r_phenotyping
#mc4r_rr['rr_1_hpo'] = [HP:
#mc4r_rr['rr_1_date'] 

,record_id,Weight-Birth kg,W-Percentile B .P,Height-Birth cm,H-Percentile Birth .P,BMI-Birth kg/m2,BMI-Percentile Birth .P,Weight - U6 kg,W-Percentile U6 .P,Height-U6 cm,H-Percentile U6 .P,BMI-U6 kg/m2,BMI-Percentile-U6 .P,Age of U6 0Y 0M,Weight-U7 kg,W-Percentile-U7.P,Height-U7 cm,H-Percentile-U7 .P,BMI-U7 kg/m2,BMI-Percentile-U7 .P,Age of U7 0Y 0M,Weight-U7a kg,W-Percentile U7a .P,Height-U7a cm,H-Percentile-U7a .P,BMI-U7a kg/m2,BMI-Percentile U7a .P,Age of U7a 0Y 0M,Weight-U8 kg,W-Percentile-U8 .P,Height-U8 cm,H-Percentile-U8 .P,BMI-U8 kg/m2,BMI-Percentile-U8 .P,Age of U8 0Y 0M,Weight-U9 kg,W-Percentile U9 .P,Height-U9 cm,H-Percentile-U9 .P,BMI-U9 kg/m2,BMI-Percentile-U9 .P,Age of U9 0Y 0M,Weight- 6y kg,W-Percentile-6y .P,Height- 6y cm,H-Percentile-6y .P,BMI 6y kg/m2,BMI-Percentile-6y .P,Weight-9y kg,W-Percentile-9y .P,Height 9y cm,H-Percentile-9y .P,BMI-9y kg/m2,BMI-Percentile-9y .P,Weight-12y kg,W-Percentile-12y .P,Height-12y cm,H-Percentile-12y .P,BMI-12y kg/m2,BMI-Percentile-12y .P,Weight-15y kg,W-Percentile-15y .P,Height-15y cm,H-Percentile-15y .P,BMI-15y kg/m2,BMI-Percentile-15y .P,Weight-18y kg,W-Percentile-18y .P,Height-18y cm,H-Percentile-18y .P,BMI -18y kg/m2,BMI-Percentile-18y .P,Weight -21y kg,Height-21y cm,BMI-21y kg/m2,BMI-Percentile-21y .P,Unnamed: 76,Sex\nm/f/d,Date of Birth DD/MM/YYYY,Age of U6 in Months,Age of U7 in Months,Age of U8 in Months,Age of U9 in Months,birth_class_obesity,birth_hpo_status,birth_date,u6_class_obesity,u6_hpo_status,u6_date,u7_class_obesity,u7_hpo_status,u7_date,u8_class_obesity,u8_hpo_status,u8_date,u9_class_obesity,u9_hpo_status,u9_date
0,1,3.485,57,50,26,13.9,83,9.370,42,72.0,19,18.1,81,00/10,16.50,99,84.0,19,23.4,>99,01/10,0,0,0,0,0,0,0,19.50,"19,5",87.0,103,44.0,"18,4",95,25.0,94,111.0,34,20.3,99,05/02,"28,9",93,"118,2",28,20.7,99,"41,4",91,"134,1",24,23.0,97,"57,5",89,"147,3",19,26.5,98,0,0,0,0,0.0,0,0,0,0.0,0,0.0,0,0,0,0,0,NaN,m,2001-08-24,10,22,0,62,HP:0001513,sct_723511001,2001-08-24,HP:0001513,sct_723511001,2002-06-24,HP:0025500,sct_410605003,2003-06-24,HP:0025501,sct_410605003,2005-08-24,HP:0025499,sct_410605003,2006-10-24
1,2,3.440,61,51,48,13.2,69,0.000,0,0.0,0,0.0,0,0,17.00,>99,91.0,91,20.5,>99,01/11,0,0,0,0,0,0,0,0.00,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0.0,0,"61,8",>99,"152,1",96,26.7,99,0,0,0,0,0.0,0,"85,2",99,168,67,30.2,99,0,0,0.0,0,0.0,0,0,0,0,0,NaN,f,2002-08-21,0,23,0,0,HP:0001513,sct_723511001,2002-08-21,HP:0001507,sct_1220561009,2003-08-21,HP:0025499,sct_410605003,2004-07-21,HP:0001507,sct_1220561009,2006-08-21,HP:0001507,sct_1220561009,2007-10-21
2,3,4.450,99,57,99,13.7,79,10.700,76,81.0,97,16.3,34,01/00,13.50,78,92.0,93,15.9,50,01/10,0,0,0,0,0,0,0,21.80,95,113.0,96,17.1,85,04/02,29.5,99,124.0,98,19.2,97,05/03,0,0,0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0.0,0,"150,6",>99,"183,5",85,44.7,>99,0,0,0.0,0,0.0,0,0,0,0,0,"letzter Termin mit 16/08: 183,5 cm, 137,5 kg, ...",m,2002-07-23,12,22,50,63,HP:0001513,sct_723511001,2002-07-23,HP:0001513,sct_723511001,2003-07-23,HP:0001513,sct_723511001,2004-05-23,HP:0025502,sct_410605003,2006-09-23,HP:0025499,sct_410605003,2007-10-23
3,4,3.450,62,51,48,13.3,72,14.500,>99,82.0,99,21.6,>99,01/00,22.20,>99,97.0,>99,23.6,>99,02/00,0,0,0,0,0,0,0,29.50,>99,112.5,99,23.3,>99,03/09,37.2,>99,124.5,>99,24.0,>99,05/00,"55,8 (07/01y)",>99,"141,5",>99,27.9,>99,"73,1 (08/05y)",>99,151,>99,32.1,>99,"124,8",>99,170,98,43.2,>99,0,0,0,0,0.0,0,0,0,0.0,0,0.0,0,0,0,0,0,NaN,f,2007-01-26,12,24,45,60,HP:0001513,sct_723511001,2007-01-26,HP:0025499,sct_410605003,2008-01-26,HP:0025500,sct_410605003,2009-01-26,HP:0025500,sct_410605003,2010-10-26,HP:0025500,sct_410605003,2012-01-26
4,5,2.700,10,48,8,11.7,24,10.000,71,75.0,49,17.8,84,00/11,13.00,33,82.0,<1,19.3,98,02/09,0,0,0,0,0,0,0,0.00,0,0.0,0,0.0,0,0,23.6,92,106.0,14,21.0,99,05/00,0,0,0,0,0.0,0,"41,4",91,"132,4",21,23.6,98,"72,3",98,154,30,30.5,99,0,0,0,0,0.0,0,0,0,0.0,0,0.0,0,0,0,0,0,NaN,f,2000-02-17,11,33,0,60,HP:0001513,sct_723511001,2000-02-17,HP:0001513,sct_723511001,2001-01-17,HP:0025499,sct_41060

### ERKER_v1_7 column names as a list


#### v1.7

In [585]:
erker_v1_7_columns= [
    # REDCap Variables
    'record_id',
    
    # 1. Formale Kriterien
    'sct_422549004','sct_399423000',
    
    # 2. Persönliche Angaben
    'sct_184099003_y', 'sct_281053000','sct_263495000','sct_315354004','sct_364699009',
    
    # 3. Patientenstatus
    'sct_278844005', 'sct_399753006', 'sct_420259009', 'sct_767023003', 'sct_184305005_rd', 'sct_16100001', 'sct_16100001_source',
    
    # 4. Behandlungsverlauf
    'sct_769681006_center','sct_440377005', 'sct_276239002_aim',
    
    # 5. Krankheitsverlauf
    'sct_424850005', 'sct_405795006_y', 'sct_405795006_m', 'sct_405795006_d', 'sct_423493009', 'sct_432213005_y', 'sct_432213005_m', 'sct_432213005_d', 'sct_717800004', 'sct_412726003', 'sct_64245008', 'sct_125679009_proposit', 'sct_842009',
    
    # 6.1 Primärdiagnose
    'sct_439401001_orpha', 'sct_439401001_orpha_sub', 'sct_439401001_icd10gm', 'sct_439401001_alphaid', 'sct_439401001_alphaidstr', 'sct_38866009', 'sct_432213005', 'sct_263493007', 'sct_116694002', 'sct_246454002', 'sct_439401001_ver', 'sct_103330002', 'sct_2931005_orpha', 'sct_2931005_icd10gm', 'sct_439401001_val', 'sct_39154008_hpo', 'sct_106221001_biom_val', 'sct_106221001_biom', 'sct_406522009', 'ln_48007_9_mitoch', 'sct_277132007___sct_277132007_263490005_260413007', 'sct_277132007___sct_416608005_1003735000_257544000_68369002_64572001', 'sct_277132007___sct_394613000', 'sct_277132007___sct_277132007_20401003_69908008', 'sct_277132007___sct_277132007_20401003_68498002', 'sct_277132007___sct_1156961008', 'sct_277132007___sct_234336002', 'sct_277132007___sct_309541009', 'sct_277132007___sct_387713003', 'sct_277132007___sct_74964007', 'sct_277132007___sct_261665006', 'sct_277132007_other',

    # 6.2 Genetischer Beufund
    'sct_439401001_gen_val','ln_ll4048_6','sct_439401001_omim_g_1', 'sct_439401001_omim_p_1', 'sct_439401001_omim_g_2', 'sct_439401001_omim_p_2', 'sct_439401001_omim_g_3', 'sct_439401001_omim_p_3',
    
        # 6.2.4 - 1. klinisch relevante Variante
        'ln_62374_4_1', 'sct_55446002_str_1', 'ln_81290_9_1', 'ln_48004_6_1', 'ln_48005_3_1', 'ln_48018_6_1', 'ln_48007_9_1', 'ln_48002_0_1', 'ln_48019_4_1', 'ln_53037_8_1', 'ln_93044_6_1',

        # 6.2.5 - 2. klinisch relevante Variante
        'ln_62374_4_2', 'sct_55446002_str_2', 'ln_81290_9_2', 'ln_48004_6_2', 'ln_48005_3_2', 'ln_48018_6_2', 'ln_48007_9_2', 'ln_48002_0_2', 'ln_48019_4_2', 'ln_53037_8_2', 'ln_93044_6_2',

        # 6.2.6 - 3. klinisch relevante Variante
        'ln_62374_4_3', 'sct_55446002_str_3', 'ln_81290_9_3', 'ln_48004_6_3', 'ln_48005_3_3', 'ln_48018_6_3', 'ln_48007_9_3', 'ln_48002_0_3', 'ln_48019_4_3', 'ln_53037_8_3', 'ln_93044_6_3',

    'sct_64195000_other',
        # 6.2.7 Genetische (Neben-)Varianten
        'ln_62374_4_4', 'sct_55446002_str_4', 'ln_81290_9_4', 'ln_48004_6_4', 'ln_48005_3_4', 'ln_48018_6_4', 'ln_48007_9_4', 
        'ln_62374_4_5', 'sct_55446002_str_5', 'ln_81290_9_5', 'ln_48004_6_5', 'ln_48005_3_5', 'ln_48018_6_5', 'ln_48007_9_5',
        'ln_62374_4_6', 'sct_55446002_str_6', 'ln_81290_9_6', 'ln_48004_6_6', 'ln_48005_3_6', 'ln_48018_6_6', 'ln_48007_9_6',
        'ln_62374_4_7', 'sct_55446002_str_7', 'ln_81290_9_7', 'ln_48004_6_7', 'ln_48005_3_7', 'ln_48018_6_7', 'ln_48007_9_7',
        'ln_62374_4_8', 'sct_55446002_str_8', 'ln_81290_9_8', 'ln_48004_6_8', 'ln_48005_3_8', 'ln_48018_6_8', 'ln_48007_9_8',

    # 6.3 Sekundärdiagnosen
        'sct_85097005_1_orpha', 'sct_85097005_1_icd10gm', 'sct_85097005_1_date', 'sct_85097005_1_ver', 'sct_85097005_2_orpha', 'sct_85097005_2_icd10gm', 'sct_85097005_2_date', 'sct_85097005_2_ver', 
        'sct_85097005_3_orpha', 'sct_85097005_3_icd10gm', 'sct_85097005_3_date', 'sct_85097005_3_ver', 'sct_85097005_4_orpha', 'sct_85097005_4_icd10gm', 'sct_85097005_4_date', 'sct_85097005_4_ver', 
        'sct_85097005_5_orpha', 'sct_85097005_5_icd10gm', 'sct_85097005_5_date', 'sct_85097005_5_ver', 'sct_85097005_6_orpha', 'sct_85097005_6_icd10gm', 'sct_85097005_6_date', 'sct_85097005_6_ver', 
        'sct_85097005_7_orpha', 'sct_85097005_7_icd10gm', 'sct_85097005_7_date', 'sct_85097005_7_ver', 'sct_85097005_8_orpha', 'sct_85097005_8_icd10gm', 'sct_85097005_8_date', 'sct_85097005_8_ver', 
        'sct_85097005_9_orpha', 'sct_85097005_9_icd10gm', 'sct_85097005_9_date', 'sct_85097005_9_ver', 'sct_85097005_10_orpha', 'sct_85097005_10_icd10gm', 'sct_85097005_10_date', 'sct_85097005_10_ver', 
        'sct_85097005_11_orpha', 'sct_85097005_11_icd10gm', 'sct_85097005_11_date', 'sct_85097005_11_ver', 'sct_85097005_12_orpha', 'sct_85097005_12_icd10gm', 'sct_85097005_12_date', 'sct_85097005_12_ver', 
        'sct_85097005_13_orpha', 'sct_85097005_13_icd10gm', 'sct_85097005_13_date', 'sct_85097005_13_ver', 'sct_85097005_14_orpha', 'sct_85097005_14_icd10gm', 'sct_85097005_14_date', 'sct_85097005_14_ver', 
        'sct_85097005_15_orpha', 'sct_85097005_15_icd10gm', 'sct_85097005_15_date', 'sct_85097005_15_ver', 'sct_85097005_16_orpha', 'sct_85097005_16_icd10gm', 'sct_85097005_16_date', 'sct_85097005_16_ver', 
        'sct_85097005_17_orpha', 'sct_85097005_17_icd10gm', 'sct_85097005_17_date', 'sct_85097005_17_ver', 'sct_85097005_18_orpha', 'sct_85097005_18_icd10gm', 'sct_85097005_18_date', 'sct_85097005_18_ver', 
        'sct_85097005_19_orpha', 'sct_85097005_19_icd10gm', 'sct_85097005_19_date', 'sct_85097005_19_ver', 'sct_85097005_20_orpha', 'sct_85097005_20_icd10gm', 'sct_85097005_20_date', 'sct_85097005_20_ver',


    # 6.4 Phänotypisierung
        'sct_8116006_1', 'sct_8116006_1_date', 'sct_8116006_1_status', 'sct_8116006_2', 'sct_8116006_2_date', 'sct_8116006_2_status', 'sct_8116006_3', 'sct_8116006_3_date', 'sct_8116006_3_status', 'sct_8116006_4', 'sct_8116006_4_date', 'sct_8116006_4_status',
        'sct_8116006_5', 'sct_8116006_5_date', 'sct_8116006_5_status', 'sct_8116006_6', 'sct_8116006_6_date', 'sct_8116006_6_status', 'sct_8116006_7', 'sct_8116006_7_date', 'sct_8116006_7_status', 'sct_8116006_8', 'sct_8116006_8_date', 'sct_8116006_8_status',
        'sct_8116006_9', 'sct_8116006_9_date', 'sct_8116006_9_status', 'sct_8116006_10', 'sct_8116006_10_date', 'sct_8116006_10_status', 'sct_8116006_11', 'sct_8116006_11_date', 'sct_8116006_11_status', 'sct_8116006_12', 'sct_8116006_12_date', 'sct_8116006_12_status',
        'sct_8116006_13', 'sct_8116006_13_date', 'sct_8116006_13_status', 'sct_8116006_14', 'sct_8116006_14_date', 'sct_8116006_14_status','sct_8116006_15', 'sct_8116006_15_date', 'sct_8116006_15_status', 'sct_8116006_16', 'sct_8116006_16_date', 'sct_8116006_16_status',
        'sct_8116006_17', 'sct_8116006_17_date', 'sct_8116006_17_status', 'sct_8116006_18', 'sct_8116006_18_date', 'sct_8116006_18_status', 'sct_8116006_19', 'sct_8116006_19_date', 'sct_8116006_19_status', 'sct_8116006_20', 'sct_8116006_20_date', 'sct_8116006_20_status',
        'sct_8116006_21', 'sct_8116006_21_date', 'sct_8116006_21_status', 'sct_8116006_22', 'sct_8116006_22_date', 'sct_8116006_22_status', 'sct_8116006_23', 'sct_8116006_23_date', 'sct_8116006_23_status', 'sct_8116006_24', 'sct_8116006_24_date', 'sct_8116006_24_status',
        'sct_8116006_25', 'sct_8116006_25_date', 'sct_8116006_25_status', 'sct_8116006_26', 'sct_8116006_26_date', 'sct_8116006_26_status', 'sct_8116006_27', 'sct_8116006_27_date', 'sct_8116006_27_status', 'sct_8116006_28', 'sct_8116006_28_date', 'sct_8116006_28_status',
        'sct_8116006_29', 'sct_8116006_29_date', 'sct_8116006_29_status', 'sct_8116006_30', 'sct_8116006_30_date', 'sct_8116006_30_status',


    # 6.5 Familienanamnese
    'sct_72705000_rd', 'sct_160430005', 'sct_13506008', 'sct_160430005_age', 'sct_66839005_rd', 'sct_160436004', 'sct_20587003', 'sct_160436004_age', 'sct_82101005_rd', 'sct_82101005_1_age', 'sct_82101005_1_gender', 'sct_82101005_2_age', 'sct_82101005_2_gender', 'sct_82101005_3_age', 'sct_82101005_3_gender', 'sct_82101005_4_age', 'sct_82101005_4_gender', 'sct_82101005_5_age', 'sct_82101005_5_gender', 'sct_82101005_6_age', 'sct_82101005_6_gender', 'sct_82101005_7_age', 'sct_82101005_7_gender', 'sct_82101005_8_age', 'sct_82101005_8_gender', 'sct_82101005_9_age', 'sct_82101005_9_gender', 'sct_82101005_10_age', 'sct_82101005_10_gender', 'sct_75226009_rd', 'sct_75226009_1_age', 'sct_75226009_1_gender', 'sct_75226009_2_age', 'sct_75226009_2_gender', 'sct_75226009_3_age', 'sct_75226009_3_gender', 'sct_75226009_4_age', 'sct_75226009_4_gender', 'sct_75226009_5_age', 'sct_75226009_5_gender', 'sct_75226009_6_age', 'sct_75226009_6_gender', 'sct_75226009_7_age', 'sct_75226009_7_gender', 'sct_75226009_8_age', 'sct_75226009_8_gender', 'sct_75226009_9_age', 'sct_75226009_9_gender', 'sct_75226009_10_age', 'sct_75226009_10_gender',

    # 7. Einwilligung Daten
    'sct_309370004_research','sct_309370004_narse',
        # 7.2.2 Varianten der Datennutzung [im NARSE]
        'sct_309370004_data', 'sct_309370004_eu', 'sct_309370004_int', 'sct_309370004_aff', 'sct_309370004_gen', 'sct_309370004_case', 'sct_123038009', 'sct_840566006', 'sct_840566006_specific',
    
    # 8. Behinderung
    'sct_21134002_class', 'sct_21134002_score',
    

    'erker_v17_complete'
]


### Create ERKER_v1.7 compatible .csv file

#### combine defined dictionaries to combined dict

In [586]:
columns_to_convert = ['OMIM_g_1', 'OMIM_g_2']
mc4r_genotyping[columns_to_convert] = mc4r_genotyping[columns_to_convert].astype("string")

In [1]:
mc4r_data_dict = mc4r_data_df.to_dict('records')
mc4r_agecat_dict = mc4r_agecat_df.to_dict('records')
mc4r_erstkontakt_dict = mc4r_erstkontakt_df.to_dict('records')
mc4r_date_diagnosis_dict = date_diagnosis_df.to_dict('records')
age_at_diagnosis_dict = age_at_diagnosis_df.to_dict('records')
consanguinity_dict = consanguinity_df.to_dict('records')
occurrence_dict = occurrence_df.to_dict('records')
zygosity_dict = zygosity_df.to_dict('records')
mc4r_genotyping_dict = mc4r_genotyping.to_dict('records')
mc4r_phenotyping_dict = mc4r_phenotyping.to_dict('records')
# Combine data from the multiple dictionaries defined above into a single list of dictionaries.
# Each dictionary corresponds to a record, and the values from different dictionaries
# are merged together into a single dictionary for each record.
mc4r_combined_dict = [
    {**row1, **row2, **row3, **row4, **row5, **row6, **row7, **row8, **row9, **row10}
    for row1, row2, row3, row4, row5, row6, row7, row8, row9, row10
    in zip(mc4r_data_dict, mc4r_agecat_dict, mc4r_erstkontakt_dict,
           mc4r_date_diagnosis_dict, age_at_diagnosis_dict, consanguinity_dict,
           occurrence_dict, zygosity_dict, mc4r_genotyping_dict, mc4r_phenotyping_dict)
]


NameError: name 'mc4r_data_df' is not defined

#### Orchestrate the mapping to combined dict

In [588]:
for row in mc4r_combined_dict:
    row['record_id'] = row['record_id']
    
   # 1. Formale Kriterien
    row['sct_422549004'] = row['Patienten ID']
    row['sct_399423000'] = '2023-09-05'

    # 2. Persönliche Angaben
    row['sct_184099003_y'] = row['Date of Birth DD/MM/YYYY'].year
    row['sct_281053000'] = 'sct_394743007'
    row['sct_281053000'] = dict_sct_263495000[row['Sex\nm/f/d']]
    try: 
        row['sct_364699009'] = dict_sct_397731000[row['Ethnic group\n']]
    except KeyError:
        pass
    
    
    # 3. Patientenstatus
    row['sct_278844005'] = 'CS_MII_Person_Vitalstatus_X'
    row['sct_767023003'] = row['Age Category']
    row['sct_184305005_rd'] = 'sct_1220561009'

    # 4. Behandlungsverlauf
    row['sct_769681006_center'] = row['Erstkontakt']
    row['sct_440377005'] = 'sct_1220561009'
    
    # 5. Krankheitsverlauf
    row['sct_424850005'] = 'sct_261665006'
    row['sct_423493009'] = row['nan_date_diagnosis']
    row['sct_432213005_y'] = row['date_diagnosis_y']
    row['sct_432213005_m'] = row['date_diagnosis_m']
    row['sct_432213005_d'] = row['date_diagnosis_d']
    row['sct_717800004'] = 'sct_1220561009'
    row['sct_64245008'] = 'sct_1220561009'
    row['sct_125679009_proposit'] = 'sct_1220561009'
    row['sct_842009'] = row['consanguinity']

    # 6.1 Primärdiagnose
    row['sct_439401001_orpha'] = 'ORPHA:71529'
    row['sct_439401001_icd10gm'] = 'E66.8'
    row['sct_439401001_alphaid'] = 'sct_1220561009'
    row['sct_432213005'] = row['DateDiagnosis']
    row['sct_263493007'] = 'sct_1220561009'
    row['sct_116694002'] = 'sct_1220561009'
    row['sct_246454002'] = row['occurrence']
    row['sct_439401001_ver'] = 'hl7_conditionverstatus_confirmed'
    row['sct_103330002'] = 'sct_373067005'
    row['sct_439401001_val'] = 'sct_439401001_20401003_405824009'
    row['sct_39154008_hpo'] = 'sct_373066001'
    row['sct_439401001_gen_biom_val'] = 'sct_1220561009'
    row['sct_406522009'] = 'sct_1220561009'
    row['ln_48007_9_mitoch'] = 'sct_1220561009'
    row['sct_277132007'] = 'sct_261665006'

    #6.2 Genetischer Befund
    row['sct_439401001_gen_val'] = 'sct_373066001' # SA099 ohne genetik befund
    row['ln_ll4048_6'] = 'sct_1220561009'
    row['sct_439401001_omim_g_1'] = row['OMIM_g_1']
    row['sct_439401001_omim_g_2'] = row['OMIM_g_2']
   
    # 1. klinisch relenvante Variante
    row['ln_62374_4_1'] =  'ln_LA26806-2'
    row['sct_55446002_str_1'] = row['hgvsfreitext_1']
    row['ln_48004_6_1'] = row['c.HGVS_1']
    row['ln_48005_3_1'] = row['p.HGVS_1']
    row['ln_48018_6_1'] = 'HGNC:6932'
    row['ln_48007_9_1'] = row['zygosity_1']
    row['ln_48002_0_1'] = row['ursprungdervariante_1']
    row['ln_48019_4_1'] = row['typeofmutation_1']
    row['ln_53037_8_1'] = row['acmg_1']
    row['ln_93044_6_1'] = 'sct_1220561009'
    
    # 2. klinisch relenvante Variante
    row['ln_62374_4_2'] =  'ln_LA26806-2'
    row['sct_55446002_str_2'] = row['hgvsfreitext_2']
    row['ln_48004_6_2'] = row['c.HGVS_2']
    row['ln_48005_3_2'] = row['p.HGVS_2']
    row['ln_48007_9_2'] = row['zygosity_2']
    row['ln_48002_0_2'] = row['ursprungdervariante_2']
    row['ln_48019_4_2'] = row['typeofmutation_2']
    row['ln_53037_8_2'] = row['acmg_2']
    
    # 3. klinisch relenvante Variante
    row['ln_62374_4_3'] =  'ln_LA26806-2'
    row['sct_55446002_str_3'] = row['hgvsfreitext_3']
    row['ln_48004_6_3'] = row['c.HGVS_3']
    row['ln_48005_3_3'] = row['p.HGVS_3']
    row['ln_48007_9_3'] = row['zygosity_3']
    row['ln_48002_0_3'] = row['ursprungdervariante_3']
    row['ln_48019_4_3'] = row['typeofmutation_3']
    row['ln_53037_8_3'] = row['acmg_3']

    
    # 6.3 Sekundärdiagnosen
    
    
    # 6.4 Phänotypisierung
    row['sct_8116006_1'] =  row['birth_class_obesity']
    row['sct_8116006_1_date'] = row['birth_date']
    row['sct_8116006_1_status'] = row['birth_hpo_status']
    row['sct_8116006_2'] = row['u6_class_obesity']
    row['sct_8116006_2_date'] = row['u6_date']
    row['sct_8116006_2_status'] = row['u6_hpo_status']
    row['sct_8116006_3'] = row['u7_class_obesity']
    row['sct_8116006_3_date'] = row['u7_date']
    row['sct_8116006_3_status'] = row['u7_hpo_status']
    row['sct_8116006_4'] = row['u8_class_obesity']
    row['sct_8116006_4_date'] = row['u8_date']
    row['sct_8116006_4_status'] = row['u8_hpo_status']
    row['sct_8116006_5'] = row['u9_class_obesity']
    row['sct_8116006_5_date'] = row['u9_date']
    row['sct_8116006_5_status'] = row['u9_hpo_status']
    
    #6.5 Familienanamnese
    row['sct_72705000_rd'] = 'sct_1220561009'
    row['sct_160430005'] = 'sct_1220561009'
    row['sct_66839005_rd'] = 'sct_1220561009'
    row['sct_160436004'] = 'sct_1220561009'
    row['sct_82101005_rd'] = 'sct_1220561009'
    row['sct_75226009_rd'] = 'sct_1220561009'
    
    # 7. Einwilligung Daten
    row['sct_309370004_research'] = 'sct_1220561009'
    row['sct_309370004_narse'] = 'sct_1220561009'
    row['sct_309370004_data'] = 'sct_1220561009'
    row['sct_309370004_eu'] = 'sct_1220561009'
    row['sct_309370004_int'] = 'sct_1220561009'
    row['sct_309370004_aff'] = 'sct_1220561009'
    row['sct_309370004_gen'] = 'sct_1220561009'
    row['sct_309370004_case'] = 'sct_1220561009'
    row['sct_123038009'] = 'sct_1220561009'
    row['sct_840566006'] = 'sct_1220561009'
    row['sct_21134002_class'] = 'sct_1220561009'
    
    pass

In [589]:
with open('/Users/adam/Documents/BIH/Adipositas/MC4R/MC4R Daten ERKER/MC4R_ERKER_v1.7.csv', mode='w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=erker_v1_7_columns, extrasaction='ignore')
    writer.writeheader()
    writer.writerows(mc4r_combined_dict) 

In [590]:
mc4r_erker_v17 = pd.read_csv('/Users/adam/Documents/BIH/Adipositas/MC4R/MC4R Daten ERKER/MC4R_ERKER_v1.7.csv', skipinitialspace=True, sep=',', decimal=',')

In [591]:
for i in range(len(mc4r_erker_v17)):
    mc4r_erker_v17.loc[i, 'sct_439401001_omim_g_1'] = f'"{mc4r_erker_v17["sct_439401001_omim_g_1"][i]}"'
    mc4r_erker_v17.loc[i, 'sct_439401001_omim_g_2'] = f'"{mc4r_erker_v17["sct_439401001_omim_g_2"][i]}"'


In [592]:
mc4r_erker_v17.to_csv('/Users/adam/Documents/BIH/Adipositas/MC4R/MC4R Daten ERKER/MC4R_ERKER_v1.7.csv', index=False)
